In [4]:
%pip install numpy pandas matplotlib seaborn scikit-learn tensorflow keras opencv-python-headless

  Using cached opencv_python_headless-4.10.0.82-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python_headless-4.10.0.82-cp37-abi3-win_amd64.whl (38.8 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\lenovo\\anaconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import cv2
import os

In [6]:
# Function to load images and labels
def load_data(dataset_path):
    categories = ['happy', 'sad', 'surprise', 'neutral', 'fear', 'disgust', 'angry']
    data = []
    labels = []

    for category in categories:
        path = os.path.join(dataset_path, category)
        class_num = categories.index(category)

        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                img_array = cv2.resize(img_array, (48, 48)) # Resize to 48x48 pixels
                data.append(img_array)
                labels.append(class_num)
            except Exception as e:
                pass

    return np.array(data), np.array(labels)

# Load training data
train_data, train_labels = load_data('dataset/train')
train_data = train_data.reshape(train_data.shape[0], 48, 48, 1)
train_data = train_data / 255.0

# Load testing data
test_data, test_labels = load_data('dataset/test')
test_data = test_data.reshape(test_data.shape[0], 48, 48, 1)
test_data = test_data / 255.0

# One-hot encode labels
train_labels = to_categorical(train_labels, num_classes=7)
test_labels = to_categorical(test_labels, num_classes=7)


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_data, train_labels, epochs=25, validation_data=(test_data, test_labels), batch_size=64)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 619,015 (2.36 MB)

 Trainable params: 619,015 (2.36 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 55s 112ms/step - accuracy: 0.2439 - loss: 1.8241 - val_accuracy: 0.3731 - val_loss: 1.6080
Epoch 2/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 46s 102ms/step - accuracy: 0.3709 - loss: 1.5972 - val_accuracy: 0.4526 - val_loss: 1.4308
Epoch 3/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 43s 95ms/step - accuracy: 0.4436 - loss: 1.4406 - val_accuracy: 0.5000 - val_loss: 1.3330
Epoch 4/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 43s 97ms/step - accuracy: 0.4836 - loss: 1.3487 - val_accuracy: 0.5174 - val_loss: 1.2786
Epoch 5/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 43s 96ms/step - accuracy: 0.5016 - loss: 1.3082 - val_accuracy: 0.5348 - val_loss: 1.2181
Epoch 6/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 43s 97ms/step - accuracy: 0.5147 - loss: 1.2761 - val_accuracy: 0.5365 - val_loss: 1.2042
Epoch 7/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 45s 100ms/step - accuracy: 0.5287 - loss: 1.2411 - val_accuracy: 0.5430 - val_loss: 1.1987
Epoch 8/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 45s 99ms/step - accuracy: 0.5385 - loss: 1.2147

In [8]:
loss, accuracy = model.evaluate(test_data, test_labels)
print(f'Test accuracy: {accuracy * 100:.2f}%')

225/225 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.7084 - loss: 0.8246
Test accuracy: 60.06%


In [9]:
model.save('emotion_detection_model.h5')

In [10]:
%pip install opencv-python-headless

  Using cached opencv_python_headless-4.10.0.82-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python_headless-4.10.0.82-cp37-abi3-win_amd64.whl (38.8 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\lenovo\\anaconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [11]:
%pip install gtts

In [12]:
import cv2
print(cv2.__version__)

4.9.0


In [1]:
# import numpy as np
# import cv2
# import pygame
# import random
# from tensorflow.keras.models import load_model
# from gtts import gTTS
# import os

# # Load your model
# model = load_model('emotion_detection_model.h5')

# # Function to predict emotion from an image
# def predict_emotion(model, img):
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     img = cv2.resize(img, (48, 48))
#     img = img.reshape(1, 48, 48, 1) / 255.0
#     prediction = model.predict(img)
#     emotion = np.argmax(prediction)
#     return emotion

# # Function to generate and play a personalized message
# def play_message(emotion):
#     emotion_dict = {
#         0: 'Aaj aapka mood bahut accha lag raha hai. Lijiye aapke mood ko aur accha karne ke liye pesh hai ek pyara sa song.',
#         1: 'Aaj aap thode udas lag rahe hain. Lijiye aapke mood ko behtar karne ke liye pesh hai ek pyara sa song.',
#         2: 'Aap bahut chakit lag rahe hain! Lijiye aapke mood ko aur accha karne ke liye pesh hai ek pyara sa song.',
#         3: 'Aap bilkul shaant lag rahe hain. Lijiye aapke mood ko behtar karne ke liye pesh hai ek pyara sa geet.',
#         4: 'Aap thoda dar rahe hain. Lijiye aapke mood ko behtar karne ke liye pesh hai ek pyara sa geet.',
#         5: 'Aapko kuch ghin aa rahi hai. Lijiye aapke mood ko behtar karne ke liye pesh hai ek pyara sa geet.',
#         6: 'Aap gusse mein lag rahe hain. Lijiye aapke mood ko behtar karne ke liye pesh hai ek pyara sa geet.'
#     }

#     message = emotion_dict[emotion]
#     tts = gTTS(text=message, lang='hi')
#     tts.save("message.mp3")
#     pygame.mixer.init()
#     pygame.mixer.music.load("message.mp3")
#     pygame.mixer.music.play()
#     while pygame.mixer.music.get_busy():
#         continue

# # Function to play a song based on the emotion
# def play_song(emotion):
#     emotion_dict = {0: 'happy', 1: 'sad', 2: 'surprise', 3: 'neutral', 4: 'fear', 5: 'disgust', 6: 'angry'}
#     songs = {
#         'happy': ['happy.mpeg', 'happy1.mpeg'],
#         'sad': ['sad.mpeg', 'sad1.mpeg'],
#         'surprise': ['sad.mpeg', 'happy.mpeg'],
#         'neutral': ['netural.mpeg', 'netural1.mpeg'],
#         'fear': ['sad1.mpeg', 'happy1.mpeg'],
#         'disgust': ['netural.mpeg', 'netural1.mpeg'],
#         'angry': ['happy.mpeg', 'happy1.']
#     }
    
#     emotion_label = emotion_dict[emotion]
#     song = random.choice(songs[emotion_label])
    
#     # Play the personalized message
#     play_message(emotion)
    
#     # Play the song
#     pygame.mixer.music.load(song)
#     pygame.mixer.music.play()

# # Start video capture
# cap = cv2.VideoCapture(0)

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break
    
#     # Define the region of interest for face detection
#     face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
#     gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)
    
#     for (x, y, w, h) in faces:
#         face = frame[y:y+h, x:x+w]
#         emotion = predict_emotion(model, face)
        
#         # Draw a rectangle around the face and label it with the emotion
#         cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
#         emotion_dict = {0: 'Happy', 1: 'Sad', 2: 'Surprise', 3: 'Neutral', 4: 'Fear', 5: 'Disgust', 6: 'Angry'}
#         emotion_text = emotion_dict[emotion]
#         cv2.putText(frame, emotion_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
        
        # Play the song for the detected emotion
#         play_song(emotion)
    
#     # Display the resulting frame
#     cv2.imshow('Emotion Detection', frame)
    
#     # Break the loop on 'q' key press
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the capture and destroy all OpenCV windows
# cap.release()
# cv2.destroyAllWindows()

# # Remove the temporary message file
# if os.path.exists("message.mp3"):
#     os.remove("message.mp3")
# hello hellow hellow 
# import numpy as np
# import cv2
# import pygame
# import random
# from tensorflow.keras.models import load_model
# from gtts import gTTS
# import os
# import time

# # Load your model
# model = load_model('emotion_detection_model.h5')

# # Function to predict emotion from an image
# def predict_emotion(model, img):
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     img = cv2.resize(img, (48, 48))
#     img = img.reshape(1, 48, 48, 1) / 255.0
#     prediction = model.predict(img)
#     emotion = np.argmax(prediction)
#     return emotion

# # Function to generate and play a personalized message
# def play_message(emotion):
#     emotion_dict = {
#         0: 'Aaj aapka mood bahut accha lag raha hai. Lijiye aapke mood ko aur accha karne ke liye pesh hai ek pyara sa song.',
#         1: 'Aaj aap thode udas lag rahe hain. Lijiye aapke mood ko behtar karne ke liye pesh hai ek pyara sa song.',
#         2: 'Aap bahut chakit lag rahe hain! Lijiye aapke mood ko aur accha karne ke liye pesh hai ek pyara sa song.',
#         3: 'Aap bilkul shaant lag rahe hain. Lijiye aapke mood ko behtar karne ke liye pesh hai ek pyara sa geet.',
#         4: 'Aap thoda dar rahe hain. Lijiye aapke mood ko behtar karne ke liye pesh hai ek pyara sa geet.',
#         5: 'Aapko kuch ghin aa rahi hai. Lijiye aapke mood ko behtar karne ke liye pesh hai ek pyara sa geet.',
#         6: 'Aap gusse mein lag rahe hain. Lijiye aapke mood ko behtar karne ke liye pesh hai ek pyara sa geet.'
#     }

#     message = emotion_dict[emotion]

    # # Ensure no other process is using the file and delete if it exists
    # if os.path.exists("message.mp3"):
    #     try:
    #         os.remove("message.mp3")
    #     except PermissionError:
    #         print("Error: Unable to delete existing 'message.mp3' file.")
    #         return

    # tts = gTTS(text=message, lang='hi')
    # tts.save("message.mp3")

    # pygame.mixer.init()
    # pygame.mixer.music.load("message.mp3")
#     pygame.mixer.music.play()
#     while pygame.mixer.music.get_busy():
#         continue

# # Function to play a song based on the emotion
# def play_song(emotion):
#     emotion_dict = {0: 'happy', 1: 'sad', 2: 'surprise', 3: 'neutral', 4: 'fear', 5: 'disgust', 6: 'angry'}
#     songs = {
#         'happy': ['happy.mpeg', 'happy1.mpeg'],
#         'sad': ['sad.mpeg', 'sad1.mpeg'],
#         'surprise': ['surprise.mpeg', 'surprise1.mpeg'],
#         'neutral': ['netural2.mpeg', 'netural3.mpeg'],
#         'fear': ['fear.mpeg', 'fear1.mpeg'],
#         'disgust': ['disgust.mpeg', 'disgust1.mpeg'],
#         'angry': ['angry.mpeg', 'angry1.mpeg']
#     }
    
#     emotion_label = emotion_dict[emotion]
#     song = random.choice(songs[emotion_label])
    
#     # Play the personalized message
#     play_message(emotion)
    
#     # Play the song
#     pygame.mixer.music.load(song)
#     pygame.mixer.music.play()

# # Start video capture
# cap = cv2.VideoCapture(0)

# # Initialize timer and state
# emotion_detected = False

# while True:
#     ret, frame = cap.read()
    # if not ret:
    #     break

    # if not emotion_detected:
    #     # Define the region of interest for face detection
    #     face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    #     gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #     faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)
        
    #     for (x, y, w, h) in faces:
    #         face = frame[y:y+h, x:x+w]
    #         emotion = predict_emotion(model, face)
            
    #         # Draw a rectangle around the face and label it with the emotion
    #         cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    #         emotion_dict = {0: 'Happy', 1: 'Sad', 2: 'Surprise', 3: 'Neutral', 4: 'Fear', 5: 'Disgust', 6: 'Angry'}
    #         emotion_text = emotion_dict[emotion]
    #         cv2.putText(frame, emotion_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
            
    #         # Play the song for the detected emotion
    #         play_song(emotion)
    #         emotion_detected = True
    
    # # Display the resulting frame
    # cv2.imshow('Emotion Detection', frame)
    
    # # Check for key press to recapture emotion
    # key = cv2.waitKey(1) & 0xFF
    # if key == ord('r'):  # Press 'r' to recapture the image and detect new emotion
    #     emotion_detected = False
    # elif key == ord('q'):  # Press 'q' to quit
#         break

# # Release the capture and destroy all OpenCV windows
# cap.release()
# cv2.destroyAllWindows()

# # Remove the temporary message file
# if os.path.exists("message.mp3"):
#     try:
#         os.remove("message.mp3")
#     except PermissionError:
#         print("Error: Unable to delete 'message.mp3' file.")



pygame 2.5.2 (SDL 2.28.3, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [1]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import cv2
import pygame
import random
from tensorflow.keras.models import load_model
from gtts import gTTS
import os
import time

# Load your model
model = load_model('emotion_detection_model.h5')

# Function to predict emotion from an image
def predict_emotion(model, img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (48, 48))
    img = img.reshape(1, 48, 48, 1) / 255.0
    prediction = model.predict(img)
    emotion = np.argmax(prediction)
    return emotion

# Function to generate and play a personalized message
def play_message(emotion):
    emotion_dict = {
        0: 'Aaj aapka mood bahut accha lag raha hai. Lijiye aapke mood ko aur accha karne ke liye pesh hai ek pyara sa song.',
        1: 'Aaj aap thode udas lag rahe hain. Lijiye aapke mood ko behtar karne ke liye pesh hai ek pyara sa song.',
        2: 'Aap bahut chakit lag rahe hain! Lijiye aapke mood ko aur accha karne ke liye pesh hai ek pyara sa song.',
        3: 'Aap bilkul shaant lag rahe hain. Lijiye aapke mood ko behtar karne ke liye pesh hai ek pyara sa geet.',
        4: 'Aap thoda dar rahe hain. Lijiye aapke mood ko behtar karne ke liye pesh hai ek pyara sa geet.',
        5: 'Aapko kuch ghin aa rahi hai. Lijiye aapke mood ko behtar karne ke liye pesh hai ek pyara sa geet.',
        6: 'Aap gusse mein lag rahe hain. Lijiye aapke mood ko behtar karne ke liye pesh hai ek pyara sa geet.'
    }

    message = emotion_dict[emotion]

    # Ensure no other process is using the file and delete if it exists
    if os.path.exists("message.mp3"):
        try:
            os.remove("message.mp3")
        except PermissionError:
            print("Error: Unable to delete existing 'message.mp3' file.")
            return

    tts = gTTS(text=message, lang='hi')
    tts.save("message.mp3")

    pygame.mixer.init()
    pygame.mixer.music.load("message.mp3")
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        continue

# Function to play a song based on the emotion
def play_song(emotion):
    emotion_dict = {0: 'happy', 1: 'sad', 2: 'surprise', 3: 'neutral', 4: 'fear', 5: 'disgust', 6: 'angry'}
    songs = {
        'happy': ['happy.mpeg'],
        'sad': ['sad.mpeg'],
        'surprise': ['surprise.mpeg'],
        'neutral': ['neutral.mpeg'],
        'fear': ['Fear.mpeg'],
        'disgust': ['Disguise.mpeg'],
        'angry': ['angry.mpeg']
    }

    emotion_label = emotion_dict[emotion]
    song = random.choice(songs[emotion_label])

    # Play the personalized message
    play_message(emotion)

    # Play the song
    pygame.mixer.music.load(song)
    pygame.mixer.music.play()

# Start video capture
cap = cv2.VideoCapture(0)

# Initialize timer and state
emotion_detected = False

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Unable to capture video")
        break

    if not emotion_detected:
        # Define the region of interest for face detection
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]
            emotion = predict_emotion(model, face)

            # Draw a rectangle around the face and label it with the emotion
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            emotion_dict = {0: 'Happy', 1: 'Sad', 2: 'Surprise', 3: 'Neutral', 4: 'Fear', 5: 'Disgust', 6: 'Angry'}
            emotion_text = emotion_dict[emotion]
            cv2.putText(frame, emotion_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

            # Play the song for the detected emotion
            play_song(emotion)
            emotion_detected = True

    # # Display the resulting frame
    # if frame is not None:
    #     cv2.imshow('Emotion Detection', frame)
    # else:
    #     print("Error: Frame is None")
     # Display the resulting frame
    cv2.imshow('Emotion Detection', frame)

    # Check for key press to recapture emotion
    key = cv2.waitKey(1) & 0xFF
    if key == ord('r'):  # Press 'r' to recapture the image and detect new emotion
        emotion_detected = False
    elif key == ord('q'):  # Press 'q' to quit
        break

# Release the capture and destroy all OpenCV windows
cap.release()
cv2.destroyAllWindows()

# Remove the temporary message file
if os.path.exists("message.mp3"):
    try:
        os.remove("message.mp3")
    except PermissionError:
        print("Error: Unable to delete 'message.mp3' file.")


pygame 2.5.2 (SDL 2.28.3, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 462ms/step


error: No file 'netural.mpeg' found in working directory 'c:\Users\lenovo\OneDrive\Desktop\emotion detect'.